This notebook assumes that you have a running instance of Neo4J Aura. You can create a free one by heading over to https://neo4j.com/

Once you have your instance, save its id as `neo4j_instance_id` and its password as `neo4j_password` as Colab secrets and grant this notebook access to said secrets.

In [1]:
!pip install neo4j yfiles_jupyter_graphs_for_neo4j pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.2 MB/s eta 0:00:00


In [2]:
from neo4j import GraphDatabase

In [3]:
from google.colab import userdata
neo4j_instance_id = userdata.get('neo4j_instance_id')
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver(f"neo4j+s://{neo4j_instance_id}.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

In [4]:
def insert_data(tx):
    query = """
    // Create Person nodes
      CREATE (alice:Person {name: 'Alice'})
      CREATE (bob:Person   {name: 'Bob'})
      CREATE (carol:Person {name: 'Carol'})
      CREATE (dave:Person  {name: 'Dave'})

      // Create Interest nodes
      CREATE (reading:Interest  {name: 'Reading'})
      CREATE (cooking:Interest  {name: 'Cooking'})
      CREATE (cycling:Interest  {name: 'Cycling'})
      CREATE (hiking:Interest   {name: 'Hiking'})
      CREATE (chess:Interest    {name: 'Chess'})
      CREATE (music:Interest    {name: 'Music'})

      // Create :LIKES relationships

      // Alice likes Reading, Cooking, Chess
      CREATE (alice)-[:LIKES]->(reading)
      CREATE (alice)-[:LIKES]->(cooking)
      CREATE (alice)-[:LIKES]->(chess)

      // Bob likes Cooking, Cycling, Music
      CREATE (bob)-[:LIKES]->(cooking)
      CREATE (bob)-[:LIKES]->(cycling)
      CREATE (bob)-[:LIKES]->(music)

      // Carol likes Reading, Hiking
      CREATE (carol)-[:LIKES]->(reading)
      CREATE (carol)-[:LIKES]->(hiking)

      // Dave likes Cycling, Chess, Music, Hiking
      CREATE (dave)-[:LIKES]->(cycling)
      CREATE (dave)-[:LIKES]->(chess)
      CREATE (dave)-[:LIKES]->(music)
      CREATE (dave)-[:LIKES]->(hiking)
      RETURN *
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(insert_data)

In [7]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='650px', width='100%'))

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [8]:
import pandas as pd

def show_jaccard_similarity(driver):
    query = """
      MATCH (p1:Person)-[:LIKES]->(i1:Interest),
        (p2:Person)-[:LIKES]->(i2:Interest)
      WHERE p1 <> p2
      WITH p1, p2,
          collect(DISTINCT i1.name) AS interests1,
          collect(DISTINCT i2.name) AS interests2
      WITH p1, p2,
          apoc.coll.intersection(interests1, interests2) AS intersection,
          apoc.coll.union(interests1, interests2) AS union
      RETURN p1.name AS PersonA,
            p2.name AS PersonB,
            CASE WHEN size(union) = 0 THEN 0
                  ELSE size(intersection)*1.0/size(union)
            END AS jaccard_similarity
      ORDER BY jaccard_similarity DESC;
    """
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

results = show_jaccard_similarity(driver)
pd.DataFrame(results)

,PersonA,PersonB,jaccard_similarity
0,Bob,Dave,0.400000
1,Dave,Bob,0.400000
2,Alice,Carol,0.250000
3,Carol,Alice,0.250000
4,Alice,Bob,0.200000
5,Bob,Alice,0.200000
6,Carol,Dave,0.200000
7,Dave,Carol,0.200000
8,Alice,Dave,0.166667
9,Dave,Alice,0.166667


In [9]:
def clear_graph(tx):
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(clear_graph)